---
title: "Three.js - 3D Animations in the Browser"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["Three.js", "Visualization", "JavaScript"]
format:
  html:
    css: ./css/three-js-demo.css
    code-fold: false
toc-depth: 4
jupyter: python3
highlight-style: github
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="importmap">
      {
          "imports": {
              "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
              "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js",
              "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js",
              "OrbitControls": "https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js",
              "Cannon": "https://cdn.jsdelivr.net/npm/cannon-es@0.20.0/dist/cannon-es.js"
          }
      }
    </script>
    <script type="module" src="./javascript/three-js-demo.js"></script>
    <script type="module" src="./javascript/three-js-grid-demo.js"></script>
    <script type="module" src="./javascript/three-js-duck-demo.js"></script>
    <script type="module" src="./javascript/three-js-many-ducks-demo.js"></script>
    <script type="module" src="./javascript/three-js-many-ducks-physics-demo.js"></script>
    <script type="module" src="./javascript/three-js-shader-demo.js"></script>
---

# Introduction

In this post we use [Three.js](https://threejs.org/) external scripts to render 3D scenes inside this Jupyter notebook. 

This post will be very similar, in terms of its stack, to the [D3.js interactive US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb).

In *that* post, we learned that Canvas and SVG are two ways in which we can display complex graphics inside a web browser. We already explored SVG graphics in [D3.js interactive US map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb). It's worth noting that a lot of the same functionality could've been replicated using the HTML Canvas element instead of SVG (which we ultimately chose for its superior interactive capabilities) -- the article mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). 

We ended up using [linkedom](https://github.com/WebReflection/linkedom#readme) to add a DOM API on top of the Deno environment. 

In fact, what I realized later on is that we could've simply used the Markdown inside our Jupyter notebook to create the `<svg>` element without the need to introduce a third-party DOM API on top of a browser-less JavaScript environment (Deno). Then, we could have simply fetched the `TopoJSON` data and constructed the map inside our external scripts with the rest of the complex D3 animations that had to be added as external scripts. 
These scripts are run by Quarto only *after* the page has been rendered to the browser, so we can easily reference the DOM elements we create inside our notes by `class` or `id`. Crucially, the external scripts are meant to run *inside the browser* (as opposed to being pre-computed in the browser-less Deno environment). Inside the browser they're able to leverage the existing DOM API provided by the browser's engine (e.g. `document.getElementById`). Hence, this way, we eliminate the need for Deno as well as Linkedom.

We will use the [IPython.display](https://ipython.readthedocs.io/en/8.26.0/api/generated/IPython.display.html) module to create HTML elements inside our notes rather than just using Markdown directly.

# Three.js, WebGL, and the Canvas Element

[Three.js](https://threejs.org/) is a library for drawing 3D graphics in the browser using JavaScript and [WebGL](https://get.webgl.org/) (see [wiki](https://www.khronos.org/webgl/wiki/Main_Page)). One excellent resource for learning WebGL is the [WebGLFundamentals](https://webglfundamentals.org/webgl/lessons) website.

WebGL runs in an HTML Canvas (i.e. `<canvas>`). 

From the WebGL wiki: 

> WebGL is a DOM API, which means that it can be used in any DOM-compatible language: e.g. JavaScript

Three.js is a library that provides conveniences in JavaScript that abstract much of this [WebGL](https://developer.mozilla.org/en-US/docs/Web/API/WebGL_API) DOM API calls behind friendlier JavaScript code. 

Note that WebGL and Canvas API are two distinct ways in which the browser draws graphics, but they both require and HTML Canvas to draw on. Three.js offers a WebGL renderer as well as a Canvas renderer. Canvas API is usually a fallback option for when WebGL, the more powerful of the two APIs, isn't available. 

Note, also, that Three.js isn't suitable for modelling purposes, for that we can use [Blender](https://www.blender.org/) or a number of other closed-source 3D modelling applications. We can even download or purchase third-party models from vendors.

Canvas and SVG elements are two ways in which we can display complex graphics inside a web browser. We already explored SVG graphics in the [D3.js interactive US Map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb) post. It's worth noting that a lot of the same functionality could've been achieved by using the HTML Canvas element with D3.js instead. The post above mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). 

However, in this post, we use Three.js (not D3.js), to draw inside an HTML Canvas element (rather than an HTML SVG element) using WebGL (as opposed to the Canvas API).

# 3D Scenes - A Primer

In *any* 3D scene, be it in the web browser, inside a game engine, in a movie, in a 3D modelling application, etc. there are a bunch of **geometries**, or **shapes** that are packaged with **materials** as **meshes**. The materials can describe simple properties like reflectivity, opacity, refraction index, etc. or they can be  **textures** which are just simple [raster](https://en.wikipedia.org/wiki/Raster_graphics) images. We can also apply **shaders** to our objects, which are complex mappings of pixels (or vertices) that make up an interesting animation.
 
A scene will also have one or more **light sources**, and a **camera** to *serve* the scene in some perspective.

## Meshes, Objects, Geometries, and Materials

An `object`, for the foreseeable future, means an object file of the [OBJ Wavefront format](https://en.wikipedia.org/wiki/Wavefront_.obj_file). These are files that consist of `object.children[n].geometry` and `object.children[n].material` fields. However, the material fields inside an OBJ are just *references* (via `usemtl` statements) to the true materials which come separately in `.mtl` files (typically from the same place as the `.obj` file).

There's usually no need to break the object down into individual child geometries and their corresponding objects. But, it's certainly possible. One use case would be if we want to apply a different transformation to each component. Usually, however, we load the object as a whole.

# Our First Scene

Let's create a `<canvas>` with `id="three-d-canvas"`.

We can do it either using raw markup below this very cell, or by using the `IPython.display` module which allows us to display rich representations of objects in a Jupyter notebook (much like the `Deno.jupyter.display` module we saw in the [D3.js US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb)).

Using `IPython.display` is more reliable.

In [13]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas'></canvas>"))

Here's the code that produces the above output. Feel free to expand and examine. We will paint the general strokes below.

<details><summary>Click to expand the Torus code</summary>
```javascript
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm'

const scene = new three.Scene();

const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

const canvas = document.getElementById("three-d-canvas")

const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
camera.position.setZ(30);

const renderer = new three.WebGLRenderer({
    canvas: canvas
});

renderer.setPixelRatio( window.devicePixelRatio );
renderer.setSize( bodyWidth, bodyHeight );


renderer.render(scene, camera);

const geometry = new three.TorusGeometry(10,3,16,100);
const material = new three.MeshBasicMaterial({ color: 0xFF6347, wireframe: true });
const torus = new three.Mesh(geometry, material);

scene.add(torus);

function animate() {
    requestAnimationFrame(animate);

    torus.rotation.x += 0.01;
    torus.rotation.y += 0.005;
    torus.rotation.z += 0.01;

    renderer.render(scene, camera);
}

animate();
```
</details>

You'll notice some general strokes.

## Creating a Scene

First, we create a `three.Scene` object.

```js
const scene = new three.Scene();
```
## Defining Scene Properties and Selecting a Canvas

Then, we define some constants for Three.js's [WebGLRenderer](https://threejs.org/docs/#api/en/renderers/WebGLRenderer) relative to the HTML document's body (grabbing the latter using the DOM API).

```js
const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;
```

We then grab the `<canvas>` element we created.

```js
const canvas = document.getElementById("three-d-canvas")
```

## Creating a Camera

After defining scene properties, we create a [PerspectiveCamera](https://threejs.org/docs/#api/en/cameras/PerspectiveCamera), supplying it the [field of view](https://en.wikipedia.org/wiki/Field_of_view) among other attributes, and setting its position.

```js
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
camera.position.setZ(30);
```
## Creating a WebGL Renderer

We then create the `WebGLRenderer` object, supplying it the `canvas` to render, as well as setting some of its parameters to the constants we defined in step one.

```js
const renderer = new three.WebGLRenderer({
    canvas: canvas
});
renderer.setPixelRatio( window.devicePixelRatio );
renderer.setSize( bodyWidth, bodyHeight );
```

### How Does a Renderer Work?

A renderer such as WebGL uses [ray-marching](https://en.wikipedia.org/wiki/Ray_marching) and [signed distance functions](https://en.wikipedia.org/wiki/Signed_distance_function) (SDFs) to render a scene. 

At a very high level, there's a plane that represents what the camera sees. This is what's displayed on our 2D screens, the plane maps to pixels on our screen. A "ray" is cast through each pixel and the ray marching algorithm determines which object in our 3D scene the ray intersects with first. This will determine which object is on the foreground and which in the background (the pixel will be colored according to this determination). 

Each point on the ray has a known coordinate in 3D space. Each object in our 3D scene also has a known position in the coordinate system (i.e. all of its vertices/edges are not unknowns). For each object, the distance between a point $p$ on the ray and the boundary of the object is calculated using a *signed distance function* (as mentioned before). These functions depend on the geometry. They are known for some primitive geometries (and their unions/intersections). A comprehensive list of SDFs is available [here](https://iquilezles.org/articles/distfunctions/). 

The point $p$ on the ray is marched only as far as the distance between the previous $p$ iterate (say $p^-$) and the closest object to it in the 3D scene (call this object $A$). This guarantees that there are no collisions with any of the *other* objects for point $p$ moving on the ray. So, $p$ is marched that far. Then, $p$ is updated again in the same way, iteratively. Eventually point $p$ touches an object (the object that intersects with the ray *first*) -- in the limit. In practice, we can stop after some $K$ iterations or when the distance $d(p, A)$ -- where $A$ is the nearest object -- is sufficiently small (i.e. less than some $\Epsilon$). 

If the ray misses, the corresponding pixel is rendered as background (for example, the color the sky). In contrast, the objects the ray hits are colored according to their distance from the pixel on the screen (according to a greyscale, for example). 

In this article we just focus on the high-level implementations in Three.js, so all of this is abstracted behind WebGL and further abstracted behind Three.js. However, in the future it may be nice to program a rudimentary renderer from scratch...

## Rendering the Scene and Adding Objects

Finally, we render the scene by invoking the renderer's `.render(scene, camera)`.

```js
renderer.render(scene, camera);
```

The `scene` object continues to be our window into the 3D world we just created. To it, we add `geometries` and their `materials` through a combination object called a [Mesh](https://threejs.org/docs/#api/en/objects/Mesh).

```js
const geometry = new three.TorusGeometry(10,3,16,100);
const material = new three.MeshBasicMaterial({ color: 0xFF6347, wireframe: true });
const torus = new three.Mesh(geometry, material);

scene.add(torus);
```

## Adding Simple Animations

We can also add a little life to the scene by using a custom animation function.
```js
function animate() {
    requestAnimationFrame(animate);

    torus.rotation.x += 0.01;
    torus.rotation.y += 0.005;
    torus.rotation.z += 0.01;

    renderer.render(scene, camera);
}
```
We invoke the animation within the outer lexical environment (for now).

```
animate();
```

## (Optional) Displaying Multiple Scenes

To display multiple 3D `scenes` in a grid, we can simply use a CSS-grid inside Jupyter.

First we lay out the markup inside the notes:

```html
<div class="three-d-grid-container">
    <div class="three-d-grid-item"><canvas id="three-d-canvas-1"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-2"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-3"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-4"></canvas></div>
</div>
```

Then, we include the stylesheet in this post's subdirectory, and use Quarto's front-matter to load it into the browser (just as we load the external scripts used to produce these rich 3D outputs):

```yaml
include-after-body:
    <script type="module" src="./javascript/three-js-demo.js"></script>
```

It turns out that there's support for including CSS inside a Quarto post. It's done using the `format.html` flag in the front-matter as follows:

```yaml
format:
  html:
    css: ./css/three-js-demo.css
```

The file `three-js-demo.css` should contain these minimal styles: 

```css
#three-d-grid-container {
    display: grid;
    grid-template-columns: repeat(2, 1fr);
    grid-template-rows: repeat(2, 1fr);
    gap: 10px;
    width: 100%;
    height: 50%;
}
.three-d-grid-item {
    display: flex;
}
```

Let's add the grid markup.

Now, inside the external script, we can reference the various `canvas` elements by `id`.

In [14]:
from IPython.display import display, HTML
display(HTML(
    """
    <div id='three-d-grid-container'>
        <div class="three-d-grid-item" id="three-d-grid-item-1"><canvas id="three-d-canvas-1"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-2"><canvas id="three-d-canvas-2"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-3"><canvas id="three-d-canvas-3"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-4"><canvas id="three-d-canvas-4"></canvas></div>
    </div>
    """
))

## Loading Custom Objects with Materials

Quarto won't serve `.obj` files (at least by default), so we can just commit the object file to the remote repository and use the `raw` GitHub link (as a CDN).

We will also need the `OBJLoader`, a Three.js add-on. This can be grabbed from a CDN as well. We may also include it in the `include-after-body.text` front matter *before* loading the script file itself as:

```yaml
<script type="importmap">
    {
        "imports": {
            "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
            "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js"
        }
    }
</script>
```
And then, within the script, import as follows:

```js
import * as three from 'three';
import { OBJLoader } from 'OBJLoader';
```

Here's the full code to render a rubber ducky which will help us in debugging our code going forward! I downloaded this model from [Sketchfab](https://sketchfab.com/3d-models/rubber-duck-ecb9ce9ff973406398ee56e391f9c902) which hosts many such free models.

### Creating a Skybox

We can also create a sky background (using what's known as a *skybox*). This can be done in several ways, the most unsophisticated of which is to add a simple gradient background. 

Going one step further, we can add a skybox (a [cubemap](https://threejs.org/docs/#api/en/loaders/CubeTextureLoader)). 

```js
const loader = new three.CubeTextureLoader();
const skyboxTexture = loader.load([
  'px.jpg', // Right
  'nx.jpg', // Left
  'py.jpg', // Top
  'ny.jpg', // Bottom
  'pz.jpg', // Front
  'nz.jpg'  // Back
]);

// Set the scene's background to `skyboxTexture`
scene.background = skyboxTexture;
```

Here's a [convenient tool](https://jaxry.github.io/panorama-to-cubemap/) that lets us convert a 360$^{\circ}$ image of a sky into a cubemap. We can also download existing cubemaps from the internet. One place to get them is [Poly Haven's HDRI section](https://polyhaven.com/a/lilienstein). We can download any sky HDRs and use [this awesome tool](https://matheowis.github.io/HDRI-to-CubeMap/) which converts the HDR file into six separate textures which can be supplied to the `loader.load()` call above.

Then, [CubeCamera](https://threejs.org/docs/#api/en/cameras/CubeCamera) is used to render the skybox images. This camera exists independently of the main camera in our scene. The `CubeCamera` uses what's known as a [render target](https://webglfundamentals.org/webgl/lessons/webgl-render-to-texture.html) (a buffer where the GPU draws pixels for a scene that is being rendered in the background). Our `CubeCamera` will use [WebGLCubeRenderTarget](https://threejs.org/docs/index.html#api/en/renderers/WebGLRenderTarget) to render the skybox images.

```js
// Create the CubeRenderTarget and CubeCamera
const cubeRenderTarget = new three.WebGLCubeRenderTarget(512);
const cubeCamera = new three.CubeCamera(1, 1000, cubeRenderTarget);

// Add CubeCamera to the scene
scene.add(cubeCamera);
```

We also have the option to use the `Sky` class that's built into Three.js to generate a procedural sky. The `Sky` class is a separate import accessible at the CDN address:

```html
<script src="https://cdn.jsdelivr.net/npm/three@0.132.2/examples/js/objects/Sky.js"></script>
```
However, this is beyond the scope of this post.

### Loading the Object\

Here's the final code snippet that produces the scene below. 

<details><summary> Click to expand the code used to generate the rubber ducky</summary>

```js
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/build/three.module.js';
import { OBJLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js';
import { MTLLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js';

// Get the container and set dimensions
const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

// Set up the canvas, scene, camera, and renderer
const canvas = document.getElementById("three-d-canvas");
const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({ canvas: canvas });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);

// Position the camera so the object will be in view.
camera.position.set(0, 20, 0);

// Add some lights so the materials are visible.
const ambientLight = new three.AmbientLight(0xffffff, 0.6);
scene.add(ambientLight);

const directionalLight = new three.DirectionalLight(0xffffff, 0.8);
directionalLight.position.set(10, 20, 10);
scene.add(directionalLight);

// Add a skybox
const loader = new three.CubeTextureLoader();
const px = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/px.png'
const nx = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/nx.png'
const py = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/py.png'
const ny = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/ny.png'
const pz = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/pz.png'
const nz = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/nz.png'
const texture = loader.load([
  px, // Right
  nx, // Left
  py, // Top
  ny, // Bottom
  pz, // Front
  nz  // Back
]);
// Set the scene's background to `texture`
scene.background = texture;
// Create the CubeRenderTarget
const cubeRenderTarget = new three.WebGLCubeRenderTarget(512);
const cubeCamera = new three.CubeCamera(1, 1000, cubeRenderTarget);
scene.add(cubeCamera);

// Instantiate MTLLoader and define the URLs for the material and object.
const mtlLoader = new MTLLoader();
const duckMtl = "https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.mtl"
const duckObj = "https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.obj"

// Load the MTL file first.
mtlLoader.load(
  duckMtl,
  (materials) => {
    materials.preload();
    // Now load the OBJ file and set its materials.
    const objLoader = new OBJLoader();
    objLoader.setMaterials(materials);
    objLoader.load(
      duckObj,
      (object) => {
        // Scale and position the loaded object.
        object.scale.set(5, 5, 5);
        object.position.set(0, 15, -20);
        scene.add(object);

        // Animation loop
        function animate() {
          requestAnimationFrame(animate);
          object.rotation.y += 0.01;
          renderer.render(scene, camera);
        }
        animate();
      },
      // onProgress callback
      (xhr) => {
        console.log("Loading object...");
      },
      // onError callback
      (error) => {
        console.error('An error occurred while loading the OBJ:', error);
      }
    );
  },
  // onProgress callback for MTL
  (xhr) => {
    console.log("Loading materials...");
  },
  // onError callback for MTL
  (error) => {
    console.error('An error occurred while loading the MTL:', error);
  }
);
```
</details>

Note that `OBJLoader` takes the URL of the object followed by a callback function that gets triggered by the object fully loading. 

The geometries of the object are in `object.children[n].geometry`. Each child is a single part of the duck (for example its eyes, wings, and nose).

In addition to the material that comes with the object, we can also add our own materials on top of it using this syntax:

```js
// Set envMap for all materials
materials.materials && Object.values(materials.materials).forEach(material => {
  material.envMap = scene.background;
  material.envMapIntensity = 0.3;  // Lower value = less reflective (range 0-1)
  material.needsUpdate = true;
});
```

The above code should be called within the `mtlLoader.load` callback function, after the `materials` have been preloaded. This uses short-circuit evaluation (logical AND) to check if `materials.materials` is truthy (i.e. not `undefined` or `null`). If it is, we can safely call `Object.values(materials.materials)` to get the values of the materials, and then method-chain to set the `envMap` and `envMapIntensity` properties for each material.

Let's create another canvas to render the rubber duck.

In [19]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-5'></canvas>"))

For the materials, we used `MTLLoader`. Another add-on downloaded from the following CDN in the `include-after-body.text` front matter:

```yaml
<script type="importmap">
    {
        "imports": {
            "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js"
        }
    }
</script>
```
Notice the nested calls? `OBJLoader` is typically called within `MTLLoader`, and gets the `material` supplied to it.

The `MTLLoader` loads the material file (`rubber_duck.mtl`) and then calls `materials.preload()`. The `OBJLoader`’s `.setMaterials(materials)` ensures that when the OBJ file is loaded, it uses the material definitions from the MTL file.

## Adding Simple Camera Controls

What good is rendering a skybox if we can't zoom and pan around to experience it in 3D? [OrbitControls](https://threejs.org/docs/#examples/en/controls/OrbitControls) allow us to do just that! Let's add them to our scene.

`OrbitControls` can be imported from the following CDN:

```html
<script src="https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js"></script>
```
As with all Three.js add-ons so far, we import `OrbitControls` using Quarto's front-matter.

Once it's imported, we add `OrbitControls` to our scene.

```js
// Add OrbitControls
const controls = new OrbitControls(camera, renderer.domElement);
controls.enableDamping = true; // Smooth damping effect during rotation
controls.dampingFactor = 0.05;
```
The line `const controls = new OrbitControls(camera, renderer.domElement)` passes `renderer.domElement` instead of just `renderer` because `OrbitControls` needs a DOM element to attach event listeners to (for mouse or touch interactions), not a renderer object. A Three.js rendering object manages the WebGL context, `renderer.domElement` is the actual HTML Canvas element to which Three.js renders the scene.

Furthermore, inside the `animate()` call, we need to ensure that we're updating `CubeCamera`'s environment map. This is done to retain realistic reflections on shiny objects. Reflective objects show surrounding environment reflected on their surfaces, so the `CubeMap` needs to be updated whenever the scene changes in order to keep reflections on objects accurate.

The animate block, which previously looked like:
```js
function animate() {
  requestAnimationFrame(animate);
  object.rotation.y += 0.01;
  renderer.render(scene, camera);
}
```
Becomes:
```js
function animate() {
  requestAnimationFrame(animate);
  object.rotation.y += 0.01;
  cubeCamera.update(renderer, scene);  
  renderer.render(scene, camera);
  controls.update();
}
```
The line `conrols.update()` is required the animation loop to ensure smooth interaction and rendering. Without this call, the camera might not respond to user input correctly.

Also, now that we have `OrbitControls`, let's ensure we're positioning the camera to show the duck at the start of our scene. This is done within the `OBJLoader` callback, where the `object` is defined.

```js
camera5.position.copy(object.position);  // Set camera at duck position
camera5.position.y += 20;  // Add y-offset (adjust value as needed)
camera5.position.z += -20  // Add z-offset offset (adjust value as needed)
camera5.lookAt(object.position) // Orient the camera to look at the duck
```

Go ahead and try the controls in the canvas above. Use:

* Right mouse button (RMB) to pan
* Left mouse button (LMB) to rotate
* Scroll wheel (SW) to zoom

Before we move on to creating more complex scenes, with multiple objects, we should explore how an OBJ file represents an object. We will need to understand the structure of an Object to manipulate it correctly using Three.js. 

# Creating More Complex Scenes

The Canvas, by itself, is a thing of wonder (as in the HTML Canvas element used in conjunction with the [Canvas API](https://developer.mozilla.org/en-US/docs/Web/API/Canvas_API)). Here's a curated list of cool [Canvas API examples](https://github.com/raphamorim/awesome-canvas?tab=readme-ov-file). For example, here's [Pong](https://cssdeck.com/labs/full/ping-pong-game-tutorial-with-html5-canvas-and-sounds). Here's a cool [Matrix animation](https://matrix.dotglitch.dev/), and here's a tool that visualizes [L-systems](https://www.kevs3d.co.uk/dev/lsystems/#). The Canvas is what makes things like drawing, vector art, and diagramming tools (such as [Lucidchart](https://www.lucidchart.comB) or [diagrams.net](https://app.diagrams.net/)) possible on the browser. Some of that burden is also, perhaps, shared by the [SVG DOM API](https://developer.mozilla.org/en-US/docs/Web/API/SVG_API) used in conjunction with an SVG element.

For more complex scenes, the HTML Canvas used purely with Canvas API is not enough. The Canvas API is usually good for 2D applications, however. It often requires explicitly setting its `context` as 2D. This is where WebGL (which is abstracted for us behind Three.js) comes into play. WebGL is a more powerful DOM API than is Canvas API.

Furthermore, for truly complex 3D web-applications (complete with mouse and keyboard controls and other advanced features), we will need to integrate Three.js into a React app, perhaps, using [React Three Fiber (r3f)](https://r3f.docs.pmnd.rs/getting-started/introduction). React Three Fiber is a library that serves as a React renderer for Three.js. It enables the creation and management of 3D scenes and objects using React components, rather than plain HTML ( allowing developers to leverage React's component system as well as state/effect management (through hooks like `useEffect`). React's lifecycle methods, like `componentDidMount`, `componentDidUpdate`, and `componentWillUnmount`, help to further abstract the browser's own DOM events (so that we don't have to use something like `DOMContentLoaded` manually).

For now, however, we will not use React Three Fiber. We will see how far we can take Three.js in Jupyter notebooks using Quarto, instead of React, to publish this page on the browser. Let's make a more complex scene now, by adding more objects. Let's add more rubber duckies to the scene! Once we are able to display more of the same object, we can perhaps explore some physics (like making it rain ducks).

## Representation of an Object

When Three.js parses an `.obj` (an OBJ file supplied to it), it represents it as a `Group` which is a JSON representation of an object that looks much like the following one (as far as Three.js is concerned): 

<details><summary>Click to expand json with comments</summary>

```json
Group {
  "children": [
    "Mesh": {
      "geometry": "BufferGeometry" // See below for full expansion...
      "material": "Material" // See below for full expansion...
    },
    "Mesh": {
      "geometry": {
        "metadata": { // Metadata provides info about the export
          "version": 4.5, // Exporter version
          "type": "BufferGeometry", // Type of object
          "generator": "BufferGeometry.toJSON"// Method that generated this JSON
        },
        "uuid": "12345678-1234-1234-1234-123456789abc", // Unique identifier for this geometry
        "type": "BufferGeometry", // Confirms that this is a BufferGeometry
        "data": { // Main container for all geometry data
          "attributes": { // Vertex attributes (data arrays for each vertex property)
            "position": { // Positions of vertices
              "itemSize": 3, // Each vertex position has 3 components: x, y, z
              "type": "Float32Array", // Data stored as a Float32Array
              "array": [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0], // Example vertex positions
              "normalized": false // Data is not normalized
            },
            "normal": { // Normals at each vertex
              "itemSize": 3, // 3 components per normal (x, y, z)
              "type": "Float32Array",
              "array": [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1], // Example normals
              "normalized": false
            },
            "uv": { // Texture coordinates
              "itemSize": 2, // Each UV coordinate has 2 components: u, v
              "type": "Float32Array",
              "array": [0, 0, 1, 0, 1, 1, 0, 1], // Example UV values
              "normalized": false
            }
          },
          "index": { // Index data defines the order to connect vertices into triangles
            "type": "Uint16Array", // Data type of the index array
            "array": [0, 1, 2, 0, 2, 3] // Indices forming triangles
          },
          "groups": [ // Groups specify portions of the geometry that use different materials
            {
              "start": 0, // Starting index in the index array for this group
              "count": 6, // Number of indices (here, one quad made of 2 triangles)
              "materialIndex": 0  // Which material from the material array should be used
            }
          ]
        }
      },
      "material": {
        "metadata": { // Metadata about this material export
          "version": 4.5, // Version of the exporter
          "type": "Object", // This is a JSON object
          "generator": "Material.toJSON" // Generated by Material.toJSON method
        },
        "uuid": "a1b2c3d4-e5f6-7890-abcd-ef1234567890", // Unique ID for the material
        "type": "MeshStandardMaterial", // Type of material (could be MeshBasicMaterial, etc.)
        "color": 16777215, // Base color (in decimal, here 16777215 equals 0xffffff - white)
        "roughness": 0.5, // Roughness parameter (controls how rough the surface is)
        "metalness": 0.5, // Metalness parameter (how metallic the material looks)
        "emissive": 0, // Emissive color (self-illumination; 0 means black, no emission)
        "opacity": 1, // Opacity value (1 means fully opaque)
        "transparent": false, // Flag indicating whether the material supports transparency
        "wireframe": false, // If true, the material renders as a wireframe instead of solid
        "side": 2 // Which side of the faces to render (2 indicates DoubleSide)
      }
    }
  ],
  // Other group properties...
}
```

</details>

This is only an example. The above JSON is a simplified version of what Three.js might output when converting an OBJ file into its internal representation. But let's go over it because the general strokes are the same...

In a `BufferGeometry`, there's am *index array*, which looks as below: 

```json
"index": {
  "type": "Uint16Array",
  "array": [0, 1, 2, 0, 2, 3]
}
```

It is an optional array that, one that defines how the *vertices* (which are stored in the `attributes.position`) are connected to form *faces* (usually composed of triangles). The values at the indices (the numbers themselves in the `index` array) refer to the same vertices in the `attributes` field of the data structure.

For example, let's say our geometry's `attributes.position` field contains the following vertices:

```json
"position": {
  "itemSize": 3,
  "array": [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]
}
```

Then, the index array above (`[0, 1, 2, 0, 2, 3]`) tells Three.js to use the vertices at positions `0, 1, 2`, in the position array, to form the first triangle, and vertices at positions `0, 2, 3` to form the second triangle. Meaning, the points $(x,y,z) = (0,0,0)$, $(1,0,0)$, and $(1,1,0)$ form a triangle, as do the points $(0,0,0), (1,1,0),$ and $(0,1,0)$. There are two triangles, so the given `Mesh`'s `geometry` represents a quad (which can also be seen from the fct that `attributes.position` contains only 4 vertices).  

The *starting index*, which is the value of `group.start` in the above object data structure refers to the *offset* in the above `index` array where a specific group (which might be assigned one *particular* material) begins. In simpler terms, it excludes certain faces from having the specified material. In this way, a `group` basically defines a subset of the index array that should be rendered with a specific material. Of course, the group also defines how many vertices, from the offset, should the material apply to (using `group.count`), as well as which material to use (using `group.materialIndex`).

Back to the [rubber ducky](./models/rubber_duck.obj) we downloaded earlier, when we look inside the `.obj` file, the groupings look different -- we see syntax like: `"o <GroupName>"` followed by the vertices. The JSON above is how Three.js interprets the `.obj`. When we print the rubbery ducky objects to the console we don't see a group of meshes (as in the JSON above), we see a single mesh. All objects in `.obj` format are translated into Three.js differently, there's a lot of variety. The `attributes.positions` and `index` arrays always behave the same way, however. The `group` is just a way to group certain vertices together, it is not a requirement of the OBJ format.

## Homogeneous Coordinates

We use $4 \times 4$ matrices (`Matrix4`), rather than $3 \times 3$ matrices, to represent transformations in 3D space. But why? 

A $3 \times 3$ matrix can handle linear transformations like rotation and scaling, but it cannot handle *affine* transformations (something as simple as a translation). All linear transformations are affine, but not all affine transformations are linear. As soon as a transformation has a transalation component, it's no longer linear (it's affine). A $3 \times 3$ matrix also can't handle projection (either *perspective* or *orthogonal* projection -- which are non-linear and linear transformations, respectively). 

The $4 \times 4$ matrix incorporates an extra row and column that stores the translation (or projection) components, enabling *all* simple transformations to be combined into one matrix. This makes it very efficient for computer graphics since we can apply a single matrix to transform an object in 3D space, reducing the number of required matrix operations. 

The use of a $4 \times 4$ matrix necessitates the use of **homogeneous coordinates** (which all 3D libraries do under the hood). This is when the normal $(x,y,x)$ coordinates are augmented as $(x,y,z,w)$ (where $w$ is an extra coordinate that's $1$ by default). Let's see how this helps.
Suppose we want to translate a point by $(t_x, t_y, t_z)$. In homogeneous coordinates, the translation matrix is:

$$
T = \begin{pmatrix}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z \\
0 & 0 & 0 & 1
\end{pmatrix}
$$

Now, take a point $p$ represented as:

$$
p = \begin{pmatrix} x \\ y \\ z \\ 1 \end{pmatrix}
$$

When we apply the translation matrix (identity transformation) to the point, we multiply:

$$
T p = \begin{pmatrix}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z \\
0 & 0 & 0 & 1
\end{pmatrix}
\begin{pmatrix} x \\ y \\ z \\ 1 \end{pmatrix}
=
\begin{pmatrix}
x + t_x \\
y + t_y \\
z + t_z \\
1
\end{pmatrix}
$$

This shows that the point $(x, y, z)$ is translated to $(x+t_x, y+t_y, z+t_z)$. We simply mentally discard the extra coordinate.

In summary, the extra coordinate $w$ in homogeneous coordinates enables us to include translation (and projection) in the *same* framework as other transformations.

## The Coordinate System

This is a good time to make an effort to understand the coordinate system in Three.js. We can actually display the axes and the coordinate grid. 

Three.js provides a couple of helper classes to visualize the coordinate system in 3D space. The most common ones are `AxesHelper` and `GridHelper`.

The `AxesHelper` displays lines for the $x$, $y$, and $z$ axes (typically colored red, green, and blue, respectively). For example:

```js
// An AxesHelper with a size of 50 units
const axesHelper = new three.AxesHelper(50);
scene.add(axesHelper);
```
This helper will render lines originating from the scene’s origin $(0, 0, 0)$, so we can visually see how objects are positioned relative to it.

If we want an additional visual cue that represents a grid on the ground (helpful for orienting objects in a scene), we can use the `GridHelper`:

```js
// A GridHelper with a grid size of 100 and 10 divisions
const gridHelper = new three.GridHelper(100, 10);
scene.add(gridHelper);
```

Finally, if our grid or axes are not showing up it may be due to the camera's position. We can point the camera to any vector! Here we point it at the origin.

```js
camera.lookAt(new three.Vector3(0, 0, 0));
```

## Adding Multiple of the Same Object to a Scene (Efficiently)

Previously we rendered separate scenes in a grid, now let's see how many ducks we can add to the same scene. 

Sure, we might think, let's just use `object.clone()` in the `OBJLoader` callback:

```js
// Clone the already loaded duck object
const duckClone = object.clone();
```

But here's where we first stumble onto major performance issues in the browser. Depending on how much memory our machine has, we will start noticing jittery behavior and slowed animations when we try to render (roughly) `50` or more of these rubber ducky objects (that's the case on my development machine, at least). Luckily, Three.js has an optimization for rendering multiple of the *same* object to the scene (even if they're animated differently). 

### Instanced Mesh

This optimization is called **instancing**. In Three.js, instancing is implemented via the `InstancedMesh` class, which is what we have to use instead of `Mesh`. `InstancedMesh` lets us render many copies of the same mesh (which, recall, is just a pairing of geometry and material) using a *single* draw call to the GPU (rather than many calls, one per each mesh). Instead of creating a separate `Mesh` for each object, which will incurs a draw call per object, we can create one `InstancedMesh`. We can assign each instance its own transformation matrix to vary the object's position, but its geometry and, to a lesser extent, materials will be fixed. This technique reduces CPU-to-GPU communication overhead, making it ideal for rendering thousands of identical objects efficiently. This is but one of many optimization techniques we can use to render thousands, or even millions of objects to a scene. Another one is using a variable *level-of-detail* (LOD) for objects that are far away from the camera or player. This technique uses shaders (which we'll discuss later) to decrease the number of vertices of our geometries if the given mesh is far away from the camera (or a player character). More on that later!

We don’t call `clone()` for each duck when using instancing. Instead, we create one `InstancedMesh` that shares the *same* geometry and material for every instance and then assigns each instance its own transformation matrix. A transformation matrix is just what it sounds like, it's a matrix supplied to `InstancedMesh` that applies a **rotation**, a **translation**, and a **scaling** transform. If this matrix sounds like it's too complicated to come up with on the spot, don't worry. We will see that its, in fact, very simple and that there are a lot of tricks to help us do just that!

`InstancedMesh` is basically an indexed data structure (a collection like an array) that stores *instances* of the mesh. These instances aren't directly accessible, unlike clones. However, we get the next best thing: an interface to update the transformation matrices of the instances (or to apply other instance attributes, like color). 

This interface consists of: 
- A getter method: `getMatrixAt(index, matrix)` 
- A setter method `setMatrixAt(index, matrix)`

One question that may arise: Why does the getter method require a `matrix` parameter? The design of the `getMatrixAt` method is such that we supply an existing `Matrix4` as an empty container. The method, then, writes the transformation matrix of the instance at index `i` (which is stored in the object JSON) into the provided dummy matrix. 

But why do we use a $4 \times 4$ matrix in 3D space? For the answer to that, refer to the subsection above on [homogeneous-coordinates](#homogeneous-coordinates). For now, let's accept this as fact. One tip, when dealing with `InstancedMecs`, is to avoid re-creating a new `Matrix4` object on every call and, instead, re-use the same dummy transformation matrix for each call.

Fist, we create an `InstancedMesh` with `N` number of ducks. The `InstancedMesh` constructor takes a single `geometry` and a single `material` object (separately). Since the duck `.obj` comes with 4 child `Meshes` (each with its own `geometry` and `material`) we need separate `InstancedMeshes` for each of the duck's parts (for each child `Mesh`, representing the eyes, beak, wings, and body of the duck). Remember, `InstancedMesh` just repeats the provided geometry and material over-and-over again (with the only difference being in the transformation matrix that's applied to each instance). 

Let's review what we need:

- We need to take each mesh of the duck and create a separate `InstancedMesh` for each of them.
  - For `N` ducks, there will be `N` beaks, `N` eyes, `N` wings, and `N` bodies.
  - For each of these `4N` meshes, we need to create a separate `InstancedMesh` object.
- We need to create `N` transformation matrices for each of the ducks.
  - This is because while we have `4N` meshes, we only need `N` transformation matrices (one for each duck). The same transformation matrix can be applied to all parts of the duck (the beak, eyes, wings, and body).

Here's the code

<details><summary>Click to expand the code used to create the instanced meshes</summary>

```js
/* Imports ---------------------------------------------- */
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/build/three.module.js';
import { OBJLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js';
import { MTLLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js';
import { OrbitControls } from "https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js";

/* Scene / camera / renderer ---------------------------------------- */
const bodyWidth  = document.getElementById("quarto-document-content").clientWidth;
const bodyHeight = 600;
const canvas6    = document.getElementById("three-d-canvas-6");

const scene6     = new three.Scene();
const camera6    = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer6  = new three.WebGLRenderer({ canvas: canvas6 });
renderer6.setPixelRatio(window.devicePixelRatio);
renderer6.setSize(bodyWidth, bodyHeight);

/* Lights */
scene6.add(new three.AmbientLight(0xffffff, 0.6));
const dir = new three.DirectionalLight(0xffffff, 0.8);
dir.position.set(10, 20, 10);
scene6.add(dir);

/* Skybox */
scene6.background = new three.CubeTextureLoader().load([
  'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/px.png',
  'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/nx.png',
  'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/py.png',
  'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/ny.png',
  'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/pz.png',
  'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/nz.png'
]);

/* Controls */
const controls = new OrbitControls(camera6, renderer6.domElement);
controls.enableDamping = true;
controls.dampingFactor = 0.05;

/* Load duck model --------------------------------------------------- */
const mtlURL = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.mtl';
const objURL = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.obj';

new MTLLoader().load(mtlURL, (materials) => {
  materials.preload();

  new OBJLoader().setMaterials(materials).load(objURL, (obj) => {

    /* Grab geometries & materials of every duck part ---------------- */
    const geos = [];
    const mats = [];
    obj.traverse((c) => { if (c.isMesh) { geos.push(c.geometry); mats.push(c.material); } });

    /* Build N shared transform matrices for each N duck *once* ----------------------------- */
    const N = 15;
    const dummy  = new three.Object3D();
    const matrices = []; // store for reuse
    for (let i = 0; i < N; i++) {
      const angle  = (i / N) * Math.PI * 2;
      const radius = 40;
      dummy.position.set(Math.cos(angle)*radius, 0, Math.sin(angle)*radius);
      dummy.rotation.y = Math.random() * Math.PI * 2;
      dummy.scale.set(3, 3, 3);
      dummy.updateMatrix();
      matrices.push(dummy.matrix.clone());   // ⬅️ keep a copy
    }

    /* One instanced mesh per part, but use *same* matrices for each of the 4 parts ------------ */
    const meshGroup = new three.Group();

    /* Adding the transformation matrix to each duck part */
    for (let p = 0; p < geos.length; p++) {
      const mesh = new three.InstancedMesh(geos[p], mats[p], N); // Make N InstancedMeshes of the duck part

      // Set the transform of each N InstancedMesh to one of the N matrices created above
      // Note that because of linearity: T(duck) = T(beak) + T(eyes) + T(body)
      // So the same transform can be applies to all parts of the duck 
      for (let i = 0; i < N; i++) {
        mesh.setMatrixAt(i, matrices[i]);    // ⬅️ reuse the matrix
      }
      mesh.instanceMatrix.needsUpdate = true;
      meshGroup.add(mesh);
    }

    scene6.add(meshGroup);

    /* Camera / animation ------------------------------------------- */
    camera6.position.set(0, 50, 100);
    camera6.lookAt(0, 0, 0);

    (function animate() {
      requestAnimationFrame(animate);
      controls.update();
      renderer6.render(scene6, camera6);
    })();
  });
});
```
</details>

Some of the steps involved are explained below.

```js
/* Build N shared transform matrices for each N duck *once* ----------------------------- */
const N = 15;
const dummy  = new three.Object3D();
const matrices = []; // store for reuse
for (let i = 0; i < N; i++) {
  const angle  = (i / N) * Math.PI * 2;
  const radius = 40;
  dummy.position.set(Math.cos(angle)*radius, 0, Math.sin(angle)*radius);
  dummy.rotation.y = Math.random() * Math.PI * 2;
  dummy.scale.set(3, 3, 3);
  dummy.updateMatrix();
  matrices.push(dummy.matrix.clone());
}
```


The step above involves a cool trick. Instead of coming up with a transformation matrix ourselves, we create a `dummy` object and use its `.position.set(x,y,z)` to manipulate the dummy object. We give it some rotation (like before). Then we do `.updateMatrix()` and use the `.matrix` method to *get* the object's transformation matrix. This way we don't have to worry about the math behind the transformation matrix. We can just use the `dummy` object to set its position, rotation, and scale.

In the example above we move the `dummy` object around a circle of `radius` 40. We subdivide the circle (of $2\pi$ radians) into `N` segments. 

```js
dummy.position.set(Math.cos(angle)*radius, 0, Math.sin(angle)*radius);
```
Sets the `dummy` object on the perimeter of the circle. 

The `dummy` object is also rotated by a random angle around its own $y$-axis.

Once we have this `dummy` object's transformation matrix, we provide it to the `.setMatrixAt(index, matrix)` method of the `InstancedMesh`.

We then create a Three.js group to capture all the `InstancedMeshes` to display them easier using the scene's `add()` method. 

```js
const meshGroup = new three.Group();
```

The line below is important. It tells Three.js that the transformation matrix of the `InstancedMesh` has changed and needs to be updated.

```js
mesh.instanceMatrix.needsUpdate = true;
```

If we don't set `needsUpdate` to `true`, Three.js won't know that the matrices have changed (because of the underlying optimizations it makes to cut on GPU workload) and won't update them in the GPU.

Finally, we need the `animate()` function to start an animation loop where we can update out `controls` for `OrbitControls`, request the animation frame, and render the scene. 

```js
(function animate() {
  requestAnimationFrame(animate);
  controls.update();
  renderer6.render(scene6, camera6);
})();
```

Expand to see the result.

<details><summary>Click to expand</summary>

In [1]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-6'></canvas>"))

</details>

## Adding Physics

Let's add some physics to our scene. We can use a popular JavaScript physics engine called [Cannon.js](https://schteppe.github.io/cannon.js/). 

First, we include Cannon.js in the `include-after-body.text` front matter, and import it from a CDN like so:

```html
<script src="https://cdn.jsdelivr.net/npm/cannon-es@0.20.0/dist/cannon-es.js"></script>
```

Like we created a `Scene` using the Three.js graphics engine, we need to create a `World` using Cannon.js. The world is where all the physics simulation happens. We can create a world like so:

```js
// Physics World
const world = new cannon.World();
world.gravity.set(0, -9.82, 0); // Gravity pointing down
```
We also need to create a ground plane, otherwise the ducks will keep falling forever. We can create a plane like so:

```js
// Ground plane
const groundBody = new cannon.Body({
  mass: 0, // Static body
  shape: new cannon.Plane(),
});
groundBody.quaternion.setFromEuler(-Math.PI / 2, 0, 0); // Rotate to be horizontal
world.addBody(groundBody);

// Ground mesh for visualization
const groundMesh = new three.Mesh(
  new three.PlaneGeometry(200, 200),
  new three.MeshStandardMaterial({ color: 0x808080, side: three.DoubleSide })
);
groundMesh.rotation.x = -Math.PI / 2;
scene6.add(groundMesh);
```
Note that we need to create a `cannon.Body` for the ground plane `Mesh`. Any object that we want to be affected by physics needs to be a `cannon.Body`. We also need to set the `quaternion` of the ground plane to be horizontal (the default is vertical). The `mass` property is set to `0`, which makes this body static. Static bodies do not move under the influence of forces or collisions—they are immovable objects in the simulation. This is ideal for objects like the ground or walls.

```js
groundBody.quaternion.setFromEuler(-Math.PI / 2, 0, 0); // Rotate to be horizontal
```

The line above sets the rotation of the ground plane to be horizontal. The `quaternion` is a way to represent rotations in 3D space. The `setFromEuler` method takes three angles (in radians) representing rotations around the $x$, $y$, and $z$ axes, respectively. In the context of 3D graphics and physics, [quaternions](https://en.wikipedia.org/wiki/Quaternion) are widely used because they provide a robust and efficient way to handle rotations. They avoid common problems associated with other rotation representations, such as [gimbal lock](https://en.wikipedia.org/wiki/Gimbal_lock#:~:text=In%20formal%20language%2C%20gimbal%20lock,which%20point%20gimbal%20lock%20occurs.), which can occur when using Euler angles. More on gimbal lock and quaternions later...

We also need to create a `cannon.Body` for each duck. We can do this in the `OBJLoader` callback, where we load the duck model. For the duck shape, we can use a `cannon.Sphere` shape for simplicity (for now). 

As before, the positions and rotations of the physics bodies are applied to the `InstancedMesh` instances using a `dummy` object.

The physics world is stepped forward using `world.step`, and the positions of the ducks are updated based on the physics simulation.


In summary:

1. **Physics World**:
   - A `cannon.World` is created with gravity pointing downward.
   - A ground plane is added to stop the ducks from falling infinitely.

2. **Duck Physics Bodies**:
   - Each duck is assigned a `cannon.Body` with a `cannon.Sphere` shape for simplicity.
   - The positions of the *physics bodies* are updated in the animation loop.

3. **Synchronizing Physics and Graphics**:
   - The positions and rotations of the physics bodies are applied to the `InstancedMesh` instances using a `dummy` object.

4. **Animation Loop**:
   - The physics world is stepped forward using `world.step`.
   - The positions of the ducks are updated based on the physics simulation.

Let's see the code:

<details><summary>Click to expand the code used to add physics</summary>

```js
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm';
import { OBJLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js';
import { MTLLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js';
import { OrbitControls } from 'https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js';
import * as cannon from 'https://cdn.jsdelivr.net/npm/cannon-es@0.20.0/dist/cannon-es.js';

/* Scene / Ccmera / renderer ---------------------------------------------- */ 
const bodyWidth = document.getElementById("quarto-document-content").clientWidth;
const bodyHeight = 600;
const canvas = document.getElementById("three-d-canvas");

const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({ canvas: canvas });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);

/* Lights ---------------------------------------------- */
scene.add(new three.AmbientLight(0xffffff, 0.6));
const dir = new three.DirectionalLight(0xffffff, 0.8);
dir.position.set(10, 20, 10);
scene.add(dir);

/* Physics World ---------------------------------------------- */
const world = new cannon.World();
world.gravity.set(0, -9.82, 0); // Gravity pointing down

// Ground plane
const groundBody = new cannon.Body({
  mass: 0, // Static body
  shape: new cannon.Plane(),
});
groundBody.quaternion.setFromEuler(-Math.PI / 2, 0, 0); // Rotate to be horizontal
world.addBody(groundBody);

// Ground mesh for visualization
const groundMesh = new three.Mesh(
  new three.PlaneGeometry(200, 200),
  new three.MeshStandardMaterial({ color: 0x808080, side: three.DoubleSide })
);
groundMesh.rotation.x = -Math.PI / 2;
scene.add(groundMesh);

/* Load duck model ---------------------------------------------- */
const mtlURL = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.mtl';
const objURL = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.obj';

new MTLLoader().load(mtlURL, (materials) => {
  materials.preload();

  new OBJLoader().setMaterials(materials).load(objURL, (obj) => {
    const geos = [];
    const mats = [];
    obj.traverse((c) => {
      if (c.isMesh) {
        geos.push(c.geometry);
        mats.push(c.material);
      }
    });

    const N = 50; // Number of ducks
    const dummy = new three.Object3D();
    const meshGroup = new three.Group();
    const duckBodies = []; // Store physics bodies

    for (let p = 0; p < geos.length; p++) {
      const mesh = new three.InstancedMesh(geos[p], mats[p], N);

      for (let i = 0; i < N; i++) {
        // Random initial positions
        const x = Math.random() * 50 - 25;
        const y = Math.random() * 50 + 50; // Start above the ground
        const z = Math.random() * 50 - 25;

        // Create a physics body for each duck
        const duckBody = new cannon.Body({
          mass: 1, // Dynamic body
          shape: new cannon.Sphere(1), // Approximate shape
          position: new cannon.Vec3(x, y, z),
        });
        world.addBody(duckBody);
        duckBodies.push(duckBody);

        // Set initial transformation for the InstancedMesh
        dummy.position.set(x, y, z);
        dummy.rotation.y = Math.random() * Math.PI * 2;
        dummy.scale.set(3, 3, 3);
        dummy.updateMatrix();
        mesh.setMatrixAt(i, dummy.matrix);
      }

      mesh.instanceMatrix.needsUpdate = true;
      meshGroup.add(mesh);
    }

    scene.add(meshGroup);

    /* Camera / Animation ---------------------------------------------- */
    camera.position.set(0, 50, 100);
    camera.lookAt(0, 0, 0);

    function animate() {
      requestAnimationFrame(animate);

      // Step the physics world
      world.step(1 / 30); // 120 FPS

      // Update the positions of the ducks
      for (let i = 0; i < N; i++) {
        const duckBody = duckBodies[i];
        const position = duckBody.position;
        const quaternion = duckBody.quaternion;

        dummy.position.set(position.x, position.y, position.z);
        dummy.quaternion.set(quaternion.x, quaternion.y, quaternion.z, quaternion.w);
        dummy.updateMatrix();

        meshGroup.children.forEach((mesh) => {
          mesh.setMatrixAt(i, dummy.matrix);
        });
      }

      meshGroup.children.forEach((mesh) => {
        mesh.instanceMatrix.needsUpdate = true;
      });

      controls.update();
      renderer.render(scene, camera);
    }

    animate();
  });
});

/* Orbit Controls ---------------------------------------------- */
const controls = new OrbitControls(camera, renderer.domElement);
controls.enableDamping = true;
controls.dampingFactor = 0.05;
```
</details>

We also added a metalic pool, feel free to see the full code [here](./javascript/three-js-many-ducks-physics-demo.js).

Finally, even though the gravitational constant is set to the same value as on Earth, the ducks appear to fall very slowly due to the scale of the scene. We can speed things up using:

```js
// Step the physics world multiple times per frame
for(let i = 0; i < 3; i++) {  // Simulate physics 3x per animation frame
    world.step(1/60);         // Standard 60 FPS physics simulation
}
```


From the docs: 

`world.step(timeStep)` takes parameter:

1. `timeStep` (required): The time step to simulate, in seconds. For example:
   - `1/60` simulates physics at 60 FPS 
   - `1/120` simulates physics at 120 FPS
   - The smaller the timestep, the more accurate but computationally expensive the simulation

There are more parameters but they are optional. 

The crucial thing to understand is that physics steps and animation frames are decoupled. The physics world is updated independently of the rendering loop. This means we can run the physics simulation at a different rate than the rendering loop. The rendering loop is variable, and based on our monitor refresh rate (usually it's 60 FPS). The physics world can be updated at a different rate than that. The key is that the physics world should be updated at a *fixed* time step. This is important for stability and accuracy in the simulation.

So when we do 3 physics steps within one animation (steps of $1/60$ seconds each in terms of time $t$ that's supplied to the physics engine), the physics simulation advances by a total of $3/60 = 1/20$ seconds. But we only show the final result after all 3 steps are done (hence why we don't need 3 corresponding animation frames). The next animation frame will start from this new state. This has the effect of speeding up our animation since each animation frame now displays the objects further along their journey (the trajectory of their motion).

Running one step of $1/20$-th second instead, as below, is inappropiate:

In [ ]:
world.step(1/20);

While mathematically $1/20 = 3*(1/60)$, they're not equivalent for physics simulation because physics engines use *numerical integration* to calculate the motion of objects. Using larger time steps (like $1/20$) leads to less accurate results because the engine has to approximate motion over a longer period. As we know, local approximation is always available but as we move far away from the known point, methods like [Taylor Series](https://en.wikipedia.org/wiki/Taylor_series) start losing accuracy. Smaller time steps $1/60$ or $1/120$ allow the engine to calculate positions and velocities more frequently, leading to more accurate simulation. With larger time steps fast-moving objects might also experience clipping issues (because their positions are updated less frequently).

In fact, the approach we used is known as [physics *sub-stepping*](https://www.unrealengine.com/es-ES/blog/physics-sub-stepping). The article goes over on how to enable sub-stepping in Unreal Engine, but the concept is the same. The idea is to run the physics simulation at a higher frequency than the rendering loop. This allows for more accurate and stable simulations, especially for fast-moving objects or complex interactions.

Let's see the result.

<details><summary>Click to expand</summary>

In [2]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-7'></canvas>"))

</details>

Of course, there are several things that are still *very* wrong with our scene: 

- The ducks shouldn't be rolling like billiards balls. We can fix that by using a more accurate shape for the ducks (instead of a sphere). For example, we can use a `cannon.ConvexPolyhedron`, which is a more accurate representation of the duck's shape. This will make the ducks fall more realistically in the above physics simulation. 
- We need to implement *buoyancy* to the water, it would be nice if the water didn't behave like the surface of a table. However, this will have to do for now.
- It would be nice if the walls reflected the ducks as well as the water's surface.

But, we will revisit this scene later and make it more believable. For now, let's move on to the next section. A section I am very excited to introduce, [Shaders](#shaders).

## Shaders

From making the surface of water realistic, to rendering realistic grass, *shaders* are the backbone of modern graphics programming. They are used for ultimate control over the rendering pipeline. Think of them as functions that map a vertex or a pixel to an RGBA color.

$$
f(x,y,z) = (x,y,z) \to (r,g,b,a)
$$

Some shaders also map a vertex to another vertex, effectively acting as a transformation function. This can help create, for example, a *displacement map* that distorts the surface of an object based on a texture (for rending things like waves in a body of water). These types of shaders look like:

$$
f(x,y,z) = (x,y,z) \to (x',y',z')
$$

Shaders are, basically, small programs (scripts) that run on the GPU and are used to control the rendering of graphics. They are written in a language called GLSL (OpenGL Shading Language). Shaders define how vertices and pixels are processed, allowing for advanced visual effects. Shaders are typically divided into two main types: **vertex shaders** and **fragment shaders**.

- **Vertex Shaders**: These shaders are responsible for processing each vertex of a 3D model. They take vertex attributes (like position, normal, and texture coordinates) and transform them into screen space. 
- **Fragment Shaders**: These shaders are responsible for processing each pixel (or fragment) of a rendered image. They determine the final color of each pixel based on various factors, such as lighting, textures, and material properties. Fragment shaders can create complex visual effects like shadows, reflections, and refractions.

### How to Use Shaders from Three.js

First, let's see how we can use shaders in Three.js. Three.js provides a `ShaderMaterial` class that allows us to create render vertex and fragment shaders. We can define our own GLSL code for both of these (the vertex and fragment shaders) and provide them to `ShaderMaterial`.

The GLSL code is typically split into two parts. Usually it's split into `<shader_name>.frag` and `<shader_name>.vert` files: the vertex shader and the fragment shader. As mentioned before, the vertex shader is responsible for transforming the vertices of the geometry, while the fragment shader is responsible for determining the color of each pixel.

First, we import the GLSL shader code into JavaScript. Importing is fancy for: creating a `waterVertexShader` and a `waterFragmentShader` string literal containing the GLSL code inside our JavaScript like so:

<details><summary>Click to expand the code used to create the water shader</summary>

```js
const waterVertexShader = `
  uniform vec3 uDepthColor;
  uniform vec3 uSurfaceColor;
  uniform float uColorOffset;
  uniform float uColorMultiplier;


  varying float vElevation;


  void main()
  {
      float mixStrenght = (vElevation  + uColorOffset)* uColorMultiplier;
      vec3 color = mix(uDepthColor,uSurfaceColor,mixStrenght) ;
      gl_FragColor = vec4(color,1.0);
  }
`;

const waterFragmentShader = `
  uniform float uTime;
  uniform float uBigWavesElevation;
  uniform vec2 uBigWavesFrequency;
  uniform float uBigWavesSpeed;

  uniform float  uSmallWavesElevation;
  uniform float  uSmallWavesFrequency;
  uniform float  uSmallWavesSpeed;
  uniform float  uSmallWavesIterations;

  varying float vElevation;

  //	Classic Perlin 3D Noise 
  //	by Stefan Gustavson
  //
  vec4 permute(vec4 x){return mod(((x*34.0)+1.0)*x, 289.0);}
  vec4 taylorInvSqrt(vec4 r){return 1.79284291400159 - 0.85373472095314 * r;}
  vec3 fade(vec3 t) {return t*t*t*(t*(t*6.0-15.0)+10.0);}

  float cnoise(vec3 P){
    vec3 Pi0 = floor(P); // Integer part for indexing
    vec3 Pi1 = Pi0 + vec3(1.0); // Integer part + 1
    Pi0 = mod(Pi0, 289.0);
    Pi1 = mod(Pi1, 289.0);
    vec3 Pf0 = fract(P); // Fractional part for interpolation
    vec3 Pf1 = Pf0 - vec3(1.0); // Fractional part - 1.0
    vec4 ix = vec4(Pi0.x, Pi1.x, Pi0.x, Pi1.x);
    vec4 iy = vec4(Pi0.yy, Pi1.yy);
    vec4 iz0 = Pi0.zzzz;
    vec4 iz1 = Pi1.zzzz;

    vec4 ixy = permute(permute(ix) + iy);
    vec4 ixy0 = permute(ixy + iz0);
    vec4 ixy1 = permute(ixy + iz1);

    vec4 gx0 = ixy0 / 7.0;
    vec4 gy0 = fract(floor(gx0) / 7.0) - 0.5;
    gx0 = fract(gx0);
    vec4 gz0 = vec4(0.5) - abs(gx0) - abs(gy0);
    vec4 sz0 = step(gz0, vec4(0.0));
    gx0 -= sz0 * (step(0.0, gx0) - 0.5);
    gy0 -= sz0 * (step(0.0, gy0) - 0.5);

    vec4 gx1 = ixy1 / 7.0;
    vec4 gy1 = fract(floor(gx1) / 7.0) - 0.5;
    gx1 = fract(gx1);
    vec4 gz1 = vec4(0.5) - abs(gx1) - abs(gy1);
    vec4 sz1 = step(gz1, vec4(0.0));
    gx1 -= sz1 * (step(0.0, gx1) - 0.5);
    gy1 -= sz1 * (step(0.0, gy1) - 0.5);

    vec3 g000 = vec3(gx0.x,gy0.x,gz0.x);
    vec3 g100 = vec3(gx0.y,gy0.y,gz0.y);
    vec3 g010 = vec3(gx0.z,gy0.z,gz0.z);
    vec3 g110 = vec3(gx0.w,gy0.w,gz0.w);
    vec3 g001 = vec3(gx1.x,gy1.x,gz1.x);
    vec3 g101 = vec3(gx1.y,gy1.y,gz1.y);
    vec3 g011 = vec3(gx1.z,gy1.z,gz1.z);
    vec3 g111 = vec3(gx1.w,gy1.w,gz1.w);

    vec4 norm0 = taylorInvSqrt(vec4(dot(g000, g000), dot(g010, g010), dot(g100, g100), dot(g110, g110)));
    g000 *= norm0.x;
    g010 *= norm0.y;
    g100 *= norm0.z;
    g110 *= norm0.w;
    vec4 norm1 = taylorInvSqrt(vec4(dot(g001, g001), dot(g011, g011), dot(g101, g101), dot(g111, g111)));
    g001 *= norm1.x;
    g011 *= norm1.y;
    g101 *= norm1.z;
    g111 *= norm1.w;

    float n000 = dot(g000, Pf0);
    float n100 = dot(g100, vec3(Pf1.x, Pf0.yz));
    float n010 = dot(g010, vec3(Pf0.x, Pf1.y, Pf0.z));
    float n110 = dot(g110, vec3(Pf1.xy, Pf0.z));
    float n001 = dot(g001, vec3(Pf0.xy, Pf1.z));
    float n101 = dot(g101, vec3(Pf1.x, Pf0.y, Pf1.z));
    float n011 = dot(g011, vec3(Pf0.x, Pf1.yz));
    float n111 = dot(g111, Pf1);

    vec3 fade_xyz = fade(Pf0);
    vec4 n_z = mix(vec4(n000, n100, n010, n110), vec4(n001, n101, n011, n111), fade_xyz.z);
    vec2 n_yz = mix(n_z.xy, n_z.zw, fade_xyz.y);
    float n_xyz = mix(n_yz.x, n_yz.y, fade_xyz.x); 
    return 2.2 * n_xyz;
  }

  void main()
  {
      vec4 modelPosition = modelMatrix * vec4(position,1.0);

      //elevation 
      float elevation = sin(modelPosition.x * uBigWavesFrequency.x + uTime * uBigWavesSpeed ) 
                      * sin(modelPosition.z * uBigWavesFrequency.y + uTime * uBigWavesSpeed)
                      * uBigWavesElevation;

      for(float i = 1.0 ; i <= uSmallWavesIterations ; i++)
      
      {
        elevation -= abs(
            cnoise(
                vec3(
                    modelPosition.xz * uSmallWavesFrequency * i,
                    uTime*uSmallWavesSpeed
                    )
                    ) * uSmallWavesElevation / i
                    );
      }

      modelPosition.y += elevation;

      vec4 viewPosition = viewMatrix * modelPosition;
      vec4 projectedPosition = projectionMatrix * viewPosition;

      gl_Position = projectedPosition;

      vElevation = elevation;
  }
`;
```

</details>

Then we can create a `ShaderMaterial` using the vertex and fragment shaders:

```js
const waterMaterial = new three.ShaderMaterial({
  uniforms: {
    time: { value: 0 },
    waterColor: { value: new three.Color(0x1E90FF) }, // Water color
    envMap: { value: scene.background }, // Environment map for reflection
  },
  vertexShader: waterVertexShader,
  fragmentShader: waterFragmentShader,
});
```

Finally, we apply the `waterMaterial` at the time of `Mesh` creation:

```js
const waterMesh = new three.Mesh(
    new three.PlaneGeometry(200, 200),
    waterMaterial
);
```

Let's render the result. 

<details><summary>Click to expand</summary>

In [3]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-8'></canvas>"))

</details>

But how does this work? Here's a [shader post](../../game_development/computer_graphics/computer_graphics.ipynb) that goes over it in detail. For us, for now, at a *very* high level:

- Vertex and fragment shaders use *uniforms* to keep track of state (between themselves as well as the outside JavaScript world)
- We run the scene (our simulation or animation) in Three.js and pass the time (from the physics simulation) to the shader using `uniforms`. Then, the shader can be a function of time...
- Position information can be passed into the shaders, for example, to achieve the effect of a surface appearing more reflective further away.

Once again, here's the full [post on shaders and GLSL](../../game_development/computer_graphics/computer_graphics.ipynb) if you're interested to explore this topic further.

## Conclusion

In this post, we explored how to create a scene with multiple ducks using Three.js. We learned how to use `InstancedMesh` to efficiently render many instances of the same geometry. We also added physics to our scene using Cannon.js, allowing the ducks to interact with the environment realistically. Finally, we introduced shaders. We used Three.js to apply a custom water shader written in GLSL. This post is part of a series on [Three.js](https://threejs.org/).